In [1]:
print('hello')

hello


In [4]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.gov.kr/search/svcmidMw?Mcode=11171&pageIndex="
result_links = []

for page in range(1, 1031):
    url = f"{base_url}{page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    items = soup.select("div.result_cont_list li.result_li_box a")
    for item in items:
        href = item.get("href")
        if href and href.startswith("/portal/service/serviceInfo/"):
            full_url = "https://www.gov.kr" + href
            result_links.append(full_url)

print(f"총 수집된 링크 개수: {len(result_links)}")
# 예: 첫 5개 링크 출력
print(result_links[:5])

총 수집된 링크 개수: 8850
['https://www.gov.kr/portal/service/serviceInfo/B19002400021', 'https://www.gov.kr/portal/service/serviceInfo/B19002400021', 'https://www.gov.kr/portal/service/serviceInfo/PTR000050187', 'https://www.gov.kr/portal/service/serviceInfo/PTR000050187', 'https://www.gov.kr/portal/service/serviceInfo/B19002400010']


In [6]:
import pandas as pd

df = pd.DataFrame(result_links, columns=["URL"])

In [12]:
pd.set_option("display.max_colwidth", None)
df.head(10)

,URL
0,https://www.gov.kr/portal/service/serviceInfo/B19002400021
1,https://www.gov.kr/portal/service/serviceInfo/B19002400021
2,https://www.gov.kr/portal/service/serviceInfo/PTR000050187
3,https://www.gov.kr/portal/service/serviceInfo/PTR000050187
4,https://www.gov.kr/portal/service/serviceInfo/B19002400010
5,https://www.gov.kr/portal/service/serviceInfo/B19002400010
6,https://www.gov.kr/portal/service/serviceInfo/PTR000050185
7,https://www.gov.kr/portal/service/serviceInfo/PTR000050926
8,https://www.gov.kr/portal/service/serviceInfo/PTR000050926
9,https://www.gov.kr/portal/service/serviceInfo/B55288100005


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def parse_detail_page(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, "html.parser")

        title_tag = soup.select_one("div#pageCont h2")
        title = title_tag.get_text(strip=True) if title_tag else ""

        if soup.select_one("div.contentsWrap.r2n"):
            content_section = soup.select_one("div.contentsWrap.r2n")
            content_texts = [
                tag.get_text(strip=True)
                for tag in content_section.select("h2, div.cont-box, p")
                if tag.get_text(strip=True)
            ]
            content = "\n".join(content_texts)

        elif soup.select_one("div.wrap_col div.left"):
            left = soup.select_one("div.wrap_col div.left")
            content = left.get_text(strip=True)

        else:
            content = "알 수 없는 레이아웃"

        return {"URL": url, "제목": title, "내용": content}
    
    except Exception as e:
        return {"URL": url, "제목": "", "내용": f"오류: {e}"}

# 테스트용: 10개만 실행
results = []
for i, url in enumerate(df["URL"][:10]):
    print(f"{i+1}/10 처리 중: {url}")
    data = parse_detail_page(url)
    results.append(data)
    time.sleep(0.5)

test_df = pd.DataFrame(results)
test_df.to_excel("test_gov_service_details.xlsx", index=False)

1/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/B19002400021
2/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/B19002400021
3/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/PTR000050187
4/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/PTR000050187
5/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/B19002400010
6/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/B19002400010
7/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/PTR000050185
8/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/PTR000050926
9/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/PTR000050926
10/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/B55288100005


In [9]:
import pandas as pd

df2 = pd.read_excel("test_gov_service_details.xlsx")
print(df2.head())

                                                 URL                    제목  \
0  https://www.gov.kr/portal/service/serviceInfo/...       기타물건 시가표준액 조사업무   
1  https://www.gov.kr/portal/service/serviceInfo/...       기타물건 시가표준액 조사업무   
2  https://www.gov.kr/portal/service/serviceInfo/...       평생학습계좌제 학습과정 안내   
3  https://www.gov.kr/portal/service/serviceInfo/...       평생학습계좌제 학습과정 안내   
4  https://www.gov.kr/portal/service/serviceInfo/...  공동주택 관리비 등 정보 공개 서비스   

                                                  내용  
0  2017년에 적용될 기타물건(시설물, 선박, 항공기, 차량 등)에 대한 시가 조사를...  
1  2017년에 적용될 기타물건(시설물, 선박, 항공기, 차량 등)에 대한 시가 조사를...  
2  평생학습계좌제 학습과정 및 교육기관 정보를 제공하는 서비스입니다.\n어떤 내용인지 ...  
3  평생학습계좌제 학습과정 및 교육기관 정보를 제공하는 서비스입니다.\n어떤 내용인지 ...  
4  공동 주택 관리비 등 투명성 확보를 위해 공동주택(※ 참조) 관리 주체가 관리비 등...  


In [14]:
pd.reset_option("display.max_colwidth")
df2

,URL,제목,내용
0,https://www.gov.kr/portal/service/serviceInfo/...,기타물건 시가표준액 조사업무,"2017년에 적용될 기타물건(시설물, 선박, 항공기, 차량 등)에 대한 시가 조사를..."
1,https://www.gov.kr/portal/service/serviceInfo/...,기타물건 시가표준액 조사업무,"2017년에 적용될 기타물건(시설물, 선박, 항공기, 차량 등)에 대한 시가 조사를..."
2,https://www.gov.kr/portal/service/serviceInfo/...,평생학습계좌제 학습과정 안내,평생학습계좌제 학습과정 및 교육기관 정보를 제공하는 서비스입니다.\n어떤 내용인지 ...
3,https://www.gov.kr/portal/service/serviceInfo/...,평생학습계좌제 학습과정 안내,평생학습계좌제 학습과정 및 교육기관 정보를 제공하는 서비스입니다.\n어떤 내용인지 ...
4,https://www.gov.kr/portal/service/serviceInfo/...,공동주택 관리비 등 정보 공개 서비스,공동 주택 관리비 등 투명성 확보를 위해 공동주택(※ 참조) 관리 주체가 관리비 등...
5,https://www.gov.kr/portal/service/serviceInfo/...,공동주택 관리비 등 정보 공개 서비스,공동 주택 관리비 등 투명성 확보를 위해 공동주택(※ 참조) 관리 주체가 관리비 등...
6,https://www.gov.kr/portal/service/serviceInfo/...,평생학습계좌제 학습이력 관리,개인의 다양한 학습이력을 온라인 학습계좌에 등록 및 관리하는 학습이력관리시스템입니다...
7,https://www.gov.kr/portal/service/serviceInfo/...,평생교육사 승급과정,평생교육사의 전문성 향상과 인적 자원을 효율적으로 활용하기 위해 운영되는 것으로 평...
8,https://www.gov.kr/portal/service/serviceInfo/...,평생교육사 승급과정,평생교육사의 전문성 향상과 인적 자원을 효율적으로 활용하기 위해 운영되는 것으로 평...
9,https://www.gov.kr/portal/service/serviceInfo/...,2주기 대학의 평생교육체제 지원사업(LiFE 2.0),□ 고등교육을 통한 성인학습자 역량 강화 \n ○ 성인 전담 학과 수와 모집인원 수...


In [11]:
pd.set_option("display.max_colwidth", None)
df2.head(10)

,URL,제목,내용
0,https://www.gov.kr/portal/service/serviceInfo/B19002400021,기타물건 시가표준액 조사업무,"2017년에 적용될 기타물건(시설물, 선박, 항공기, 차량 등)에 대한 시가 조사를 통해 행정안전부의 시가표준액 조정기준 수립을 지원하여 과세 형평성 제고에 기여\n어떤 내용인지 궁금하시죠?\n지원형태정보제공지원내용○ 기타물건 종별 시가표준액(안) 제공- 행정안전부 기타물건 시가표준액 조정기준 수립 시 종별 시가표준액 기초자료 제공○ 개별, 대량 산정모형 및 연도별 평가 모형 기준(안) 연구- 시설물 등 기타물건에 대한 종별, 물건별 적용을 위한 시가표준액 개별, 대량 산정모형(안) 연구 및 제시- 내용 연수 및 잔존가치율 적용 등을 위한 연도별 평가 모형(안) 연구 및 제시\n지원형태\n지원내용\n이런 분들께 해당합니다.\n지원대상전체\n지원대상\n이용 방법은 이렇습니다.\n접수기관공시통계본부 공시기획처 공시통계총괄부 / 연락처 053-663-8209\n접수기관\n※ 자세한 사항은 해당 기관의 홈페이지 또는 문의처를 통해 확인하시기 바랍니다.\n참고정보입니다.\n근거법령[법령] 지방세법(제4조, 제3항)소관기관한국부동산원최종수정일2025.02.04\n근거법령\n소관기관\n최종수정일\n분야별 서비스 Best 5\n기관별 서비스 Best 5"
1,https://www.gov.kr/portal/service/serviceInfo/B19002400021,기타물건 시가표준액 조사업무,"2017년에 적용될 기타물건(시설물, 선박, 항공기, 차량 등)에 대한 시가 조사를 통해 행정안전부의 시가표준액 조정기준 수립을 지원하여 과세 형평성 제고에 기여\n어떤 내용인지 궁금하시죠?\n지원형태정보제공지원내용○ 기타물건 종별 시가표준액(안) 제공- 행정안전부 기타물건 시가표준액 조정기준 수립 시 종별 시가표준액 기초자료 제공○ 개별, 대량 산정모형 및 연도별 평가 모형 기준(안) 연구- 시설물 등 기타물건에 대한 종별, 물건별 적용을 위한 시가표준액 개별, 대량 산정모형(안) 연구 및 제시- 내용 연수 및 잔존가치율 적용 등을 위한 연도별 평가 모형(안) 연구 및 제시\n지원형태\n지원내용\n이런 분들께 해당합니다.\n지원대상전체\n지원대상\n이용 방법은 이렇습니다.\n접수기관공시통계본부 공시기획처 공시통계총괄부 / 연락처 053-663-8209\n접수기관\n※ 자세한 사항은 해당 기관의 홈페이지 또는 문의처를 통해 확인하시기 바랍니다.\n참고정보입니다.\n근거법령[법령] 지방세법(제4조, 제3항)소관기관한국부동산원최종수정일2025.02.04\n근거법령\n소관기관\n최종수정일\n분야별 서비스 Best 5\n기관별 서비스 Best 5"
2,https://www.gov.kr/portal/service/serviceInfo/PTR000050187,평생학습계좌제 학습과정 안내,"평생학습계좌제 학습과정 및 교육기관 정보를 제공하는 서비스입니다.\n어떤 내용인지 궁금하시죠?\n지원형태정보제공지원내용ㅇ 평생학습계좌제 학습과정 정보 검색 및 조회- 강좌명, 교육내용, 교육시간, 교육기간 등 학습과정 상세 정보ㅇ 평생학습계좌제 교육기관 정보 검색 및 조회- 기관명, 주소, 전화번호, 홈페이지 등 교육기관 상세 정보ㅇ 검정고시 연계 학습과정 정보 검색 및 조회- 평가인정 학습과정의 검정고시 연계 과목 및 학력수준 정보\n지원형태\n지원내용\n이런 분들께 해당합니다.\n지원대상전 국민\n지원대상\n이용 방법은 이렇습니다.\n접수기관학습플랫폼운영실 / 연락처 02-3780-9965\n접수기관\n※ 자세한 사항은 해당 기관의 홈페이지 또는 문의처를 통해 확인하시기 바랍니다.\n참고정보입니다.\n근거법령[법령] 평생교육법(제23조)[법령] 평생교육법 시행령(제14조의2)소관기관국가평생교육진흥원 디지털평생교육본부 학습플랫폼운영실최종수정일2025.01.22\n근거법령\n소관기관\n최종수정일\n분야별 서비스 Best 5\n기관별 서비스 Best 5"
3,https://www.gov.kr/portal/service/serviceInfo/PTR000050187,평생학습계좌제 학습과정 안내,"평생학습계좌제 학습과정 및 교육기관 정보를 제공하는 서비스입니다.\n어떤 내용인지 궁금하시죠?\n지원형태정보제공지원내용ㅇ 평생학습계좌제 학습과정 정보 검색 및 조회- 강좌명, 교육내용, 교육시간, 교육기간 등 학습과정 상세 정보ㅇ 평생학습계좌제 교육기관 정보 검색 및 조회- 기관명, 주소, 전화번호, 홈페이지 등 교육기관 상세 정보ㅇ 검정고시 연계 학습과정 정보 검색 및 조회- 평가인정 학습과정의 검정고시 연계 과목 및 학력수준 정보\n지원형태\n지원내용\n이런 분들께 해당합니다.\n지원대상전 국민\n지원대상\n이용 방법은 이렇습니다.\n접수기관학습플랫폼운영실 / 연락처 02-3780-9965\n접수기관\n※ 자세한 사항은 해당 기관의 홈페이지 또는 문의처를 통해 확인하시기 바랍니다.\n참고정보입니다.\n근거법령[법령] 평생교육법(제23조)[법령] 평생교육법 시행령(제14조의2)소관기관국가평생교육진흥원 디지털평생교육본부 학습플랫폼운영실최종수정일2025.01.22\n근거법령\n소관기관\n최종수정일\n분야별 서비스 Best 5\n기관별 서비스 Best 5"
4,https://www.gov.kr/portal/service/serviceInfo/B19002400010,공동주택 관리비 등 정보 공개 서비스,"공동 주택 관리비 등 투명성 확보를 위해 공동주택(※ 참조) 관리 주체가 관리비 등 내용을 공개하는 제도\n※ 의무 공개 단지 : 300세대, 150세대 이상으로서 승강기가 설치되었거나 중앙 집중식 난방방식(지역난방방식 포함) 또는 150세대 이상인 주상복합 공동주택\n어떤 내용인지 궁금하시죠?\n지원형태정보제공지원내용○ 관리비 및 에너지 사용량 공개- 관리비 등 47개 항목, 에너지 사용량 공개ㆍ비교를 통한 관리 투명성 제고○ 입찰공고 및 결과 공개- 공동주택 관리 업자 선정 및 구매ㆍ공사 등 입찰시스템 구축 제공 및 입찰 결과 공개를 통한 비리 차단 기여○ 외부 회계감사 보고서 공개- 최근 의무화된 단지별 외부 회계감사보고서를 공개○ 시설물 유지관리 이력 정보 공개- 공사 내용, 위치, 기간, 계약 금액 등 유지관리 이력 공개를 통한 주택 품질 강화 및 장수명화 기여○ 단지 기본 정보 공개- 사용승인일, 단지 유형, 관리비 부과 면적, 동수, 세대수 등 11개 항목 공개\n지원형태\n지원내용\n이런 분들께 해당합니다.\n지원대상전 국민\n지원대상\n이용 방법은 이렇습니다.\n접수기관부동산시장관리본부 시장관리처 K-apt관리단 / 연락처 053-663-8766\n접수기관\n※ 자세한 사항은 해당 기관의 홈페이지 또는 문의처를 통해 확인하시기 바랍니다.\n참고정보입니다.\n근거법령[법령] 공동주택관리법(제23조)[법령] 공동주택관리법(제88조)소관기관한국부동산원최종수정일2025.04.30\n근거법령\n소관기관\n최종수정일\n분야별 서비스 Best 5\n기관별 서비스 Best 5"
5,https://www.gov.kr/portal/service/serviceInfo/B19002400010,공동주택 관리비 등 정보 공개 서비스,"공동 주택 관리비 등 투명성 확보를 위해 공동주택(※ 참조) 관리 주체가 관리비 등 내용을 공개하는 제도\n※ 의무 공개 단지 : 300세대, 150세대 이상으로서 승강기가 설치되었거나 중앙 집중식 난방방식(지역난방방식 포함) 또는 150세대 이상인 주상복합 공동주택\n어떤 내용인지 궁금하시죠?\n지원형태정보제공지원내용○ 관리비 및 에너지 사용량 공개- 관리비 등 47개 항목, 에너지 사용량 공개ㆍ비교를 통한 관리 투명성 제고○ 입찰공고 및 결과 공개- 공동주택 관리 업자 선정 및 구매ㆍ공사 등 입찰시스템 구축 제공 및 입찰 결과 공개를 통한 비리 차단 기여○ 외부 회계감사 보고서 공개- 최근 의무화된 단지별 외부 회계감사보고서를 공개○ 시설물 유지관리 이력 정보 공개- 공사 내용, 위치, 기간, 계약 금액 등 유지관리 이력 공개를 통한 주택 품질 강화 및 장수명화 기여○ 단지 기본 정보 공개- 사용승인일, 단지 유형

In [15]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.gov.kr/search/svcmidMw?Mcode=11171&pageIndex="
result_links = []

for page in range(1, 2):
    url = f"{base_url}{page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    items = soup.select("li.result_li_box span.list_font17 a")
    for item in items:
        href = item.get("href")
        if href and href.startswith("/portal/service/serviceInfo/"):
            full_url = "https://www.gov.kr" + href
            result_links.append(full_url)

print(f"총 수집된 링크 개수: {len(result_links)}")
print(result_links[:5])

총 수집된 링크 개수: 9
['https://www.gov.kr/portal/service/serviceInfo/B19002400021', 'https://www.gov.kr/portal/service/serviceInfo/PTR000050187', 'https://www.gov.kr/portal/service/serviceInfo/B19002400010', 'https://www.gov.kr/portal/service/serviceInfo/PTR000050185', 'https://www.gov.kr/portal/service/serviceInfo/PTR000050926']


In [16]:
import pandas as pd

df3 = pd.DataFrame(result_links, columns=["URL"])

In [20]:
pd.set_option("display.max_colwidth", None)
df3.head(10)

,URL
0,https://www.gov.kr/portal/service/serviceInfo/B19002400021
1,https://www.gov.kr/portal/service/serviceInfo/PTR000050187
2,https://www.gov.kr/portal/service/serviceInfo/B19002400010
3,https://www.gov.kr/portal/service/serviceInfo/PTR000050185
4,https://www.gov.kr/portal/service/serviceInfo/PTR000050926
5,https://www.gov.kr/portal/service/serviceInfo/B55288100005
6,https://www.gov.kr/portal/service/serviceInfo/PTR000050927
7,https://www.gov.kr/portal/service/serviceInfo/PTR000050928
8,https://www.gov.kr/portal/service/serviceInfo/B55201600001


In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.gov.kr/search/svcmidMw?Mcode=11171&pageIndex=1"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

result_links = []
items = soup.select("li.result_li_box a")

for item in items:
    href = item.get("href")
    if href and ("/portal/service/serviceInfo/" in href or "/mw/AA020InfoCappView.do" in href):
        full_url = "https://www.gov.kr" + href
        result_links.append(full_url)

# 중복 제거
result_links = list(set(result_links))

# DataFrame 생성
df3 = pd.DataFrame(result_links, columns=["URL"])
print(df3)

                                                                                         URL
0                                 https://www.gov.kr/portal/service/serviceInfo/PTR000050928
1                                 https://www.gov.kr/portal/service/serviceInfo/B55201600001
2  https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A04002&CappBizCD=00000000001&tp_seq=
3                                 https://www.gov.kr/portal/service/serviceInfo/PTR000050926
4                                 https://www.gov.kr/portal/service/serviceInfo/PTR000050187
5                                 https://www.gov.kr/portal/service/serviceInfo/B55288100005
6                                 https://www.gov.kr/portal/service/serviceInfo/B19002400021
7                                 https://www.gov.kr/portal/service/serviceInfo/PTR000050927
8                                 https://www.gov.kr/portal/service/serviceInfo/B19002400010
9                                 https://www.gov.kr/portal/service/se

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

base_url = "https://www.gov.kr/search/svcmidMw?Mcode=11171&pageIndex="
result_links = []

for page in range(1, 1031):
    url = f"{base_url}{page}"
    print(f"{page} 페이지 처리 중...")
    
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, "html.parser")

        items = soup.select("li.result_li_box a")
        for item in items:
            href = item.get("href")
            if href and ("/portal/service/serviceInfo/" in href or "/mw/AA020InfoCappView.do" in href):
                full_url = "https://www.gov.kr" + href
                result_links.append(full_url)

        time.sleep(0.5)  # 서버 과부하 방지용

    except Exception as e:
        print(f"{page} 페이지 에러: {e}")
        continue

# 중복 제거
result_links = list(set(result_links))

# DataFrame 생성 및 저장
df = pd.DataFrame(result_links, columns=["URL"])
df.to_excel("gov_all_links.xlsx", index=False)
print(f"총 수집된 링크 수: {len(df)}")

1 페이지 처리 중...
2 페이지 처리 중...
3 페이지 처리 중...
4 페이지 처리 중...
5 페이지 처리 중...
6 페이지 처리 중...
7 페이지 처리 중...
8 페이지 처리 중...
9 페이지 처리 중...
10 페이지 처리 중...
11 페이지 처리 중...
12 페이지 처리 중...
13 페이지 처리 중...
14 페이지 처리 중...
15 페이지 처리 중...
16 페이지 처리 중...
17 페이지 처리 중...
18 페이지 처리 중...
19 페이지 처리 중...
20 페이지 처리 중...
21 페이지 처리 중...
22 페이지 처리 중...
23 페이지 처리 중...
24 페이지 처리 중...
25 페이지 처리 중...
26 페이지 처리 중...
27 페이지 처리 중...
28 페이지 처리 중...
29 페이지 처리 중...
30 페이지 처리 중...
31 페이지 처리 중...
32 페이지 처리 중...
33 페이지 처리 중...
34 페이지 처리 중...
35 페이지 처리 중...
36 페이지 처리 중...
37 페이지 처리 중...
38 페이지 처리 중...
39 페이지 처리 중...
40 페이지 처리 중...
41 페이지 처리 중...
42 페이지 처리 중...
43 페이지 처리 중...
44 페이지 처리 중...
45 페이지 처리 중...
46 페이지 처리 중...
47 페이지 처리 중...
48 페이지 처리 중...
49 페이지 처리 중...
50 페이지 처리 중...
51 페이지 처리 중...
52 페이지 처리 중...
53 페이지 처리 중...
54 페이지 처리 중...
55 페이지 처리 중...
56 페이지 처리 중...
57 페이지 처리 중...
58 페이지 처리 중...
59 페이지 처리 중...
60 페이지 처리 중...
61 페이지 처리 중...
62 페이지 처리 중...
63 페이지 처리 중...
64 페이지 처리 중...
65 페이지 처리 중...
66 페이지 처리 중...
67 페이지 처리 중...
68 페

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 엑셀 파일에서 링크 불러오기
df = pd.read_excel("gov_all_links.xlsx")

def parse_detail_page(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, "html.parser")

        title_tag = soup.select_one("div#pageCont h2")
        title = title_tag.get_text(strip=True) if title_tag else ""

        if soup.select_one("div.contentsWrap.r2n"):
            content_section = soup.select_one("div.contentsWrap.r2n")
            content_texts = [
                tag.get_text(strip=True)
                for tag in content_section.select("h2, div.cont-box, p")
                if tag.get_text(strip=True)
            ]
            content = "\n".join(content_texts)

        elif soup.select_one("div.wrap_col div.left"):
            left = soup.select_one("div.wrap_col div.left")
            content = left.get_text(strip=True)

        else:
            content = "알 수 없는 레이아웃"

        return {"URL": url, "제목": title, "내용": content}
    
    except Exception as e:
        return {"URL": url, "제목": "", "내용": f"오류: {e}"}

# 앞에서 10개만 처리
results = []
for i, url in enumerate(df["URL"][:10]):
    print(f"{i+1}/10 처리 중: {url}")
    data = parse_detail_page(url)
    results.append(data)
    time.sleep(0.5)

# 결과 저장
test_df = pd.DataFrame(results)
test_df.to_excel("test_gov_service_details.xlsx", index=False)

1/10 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A03004&CappBizCD=15701000076&tp_seq=
2/10 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A08003&CappBizCD=13200000052&tp_seq=01
3/10 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A09002&CappBizCD=12100000169&tp_seq=
4/10 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A02002&CappBizCD=14800000063&tp_seq=01
5/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/403000000147
6/10 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A03007&CappBizCD=15200000480&tp_seq=01
7/10 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A03006&CappBizCD=14100000069&tp_seq=01
8/10 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A09002&CappBizCD=12200000235&tp_seq=01
9/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/508000000565
10/10 처리 중: https://www.gov.kr/portal/service/serviceInfo/356000000232


In [2]:
# 특수문자 제거
test_df["내용"] = test_df["내용"].str.replace(r"[\n\r\t]", "", regex=True)

# 고정 텍스트 제거
제거문구 = "근거법령소관기관최종수정일분야별 서비스 Best 5기관별 서비스 Best 5"
test_df["내용"] = test_df["내용"].str.replace(제거문구, "", regex=False)

In [3]:
# 이 페이지에 만족하시나요?네아니오 문구 이후 제거

# 기준 문구 지정
cut_point = "이 페이지에 만족하시나요?네아니오"

# 문구 이후 제거
test_df["내용"] = test_df["내용"].apply(
    lambda x: x.split(cut_point)[0] if isinstance(x, str) and cut_point in x else x
)

In [5]:
pd.set_option('display.max_colwidth', None)

In [6]:
test_df

,URL,제목,내용
0,https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A03004&CappBizCD=15701000076&tp_seq=,"정보통신기술자(인정,등급변경,경력인정, 전자형변경)신청","서비스 개요신청방법방문신청자격누구나 신청 가능처리기간총 7일처리기간 계산 방법신청서정보통신기술자(인정,등급변경,경력인정, 전자형 변경) 신청서 (정보통신공사업법 시행에 관한 규정 : 별지서식 30호)구비서류있음 (하단 참조)수수료정보통신공사업법 제57조제1항 관련 수수료기본정보제공 내용이 민원은 정보통신공사업법 제39조 및 같은 법 시행령 제39조에 따라 정보통신기술자로 인정받고자 하는 자가 해당 서류(자격증 등)을 구비하여 위탁기관인 한국정보통신공사협회에 신청신청 방법 및 절차1접수한국정보통신공사협회2처리한국정보통신공사협회참고각 기관을 선택하면 해당기관 정보조회가 가능하며, 조회된 접수/처리기관에 대한 실제 민원 접수/처리 가능여부는 해당기관에 확인해 주시기 바랍니다.제출 서류민원인이 제출해야하는 서류사진(2.5cm*3cm) 2매(등급변경은 1매)별지 제2호서식의 경력확인서(재직사실을 증명하는 서류가 첨부된 경우에 한함)정보통신기술자 경력수첩(등급변경 및 경력인정에 한함)신분증 원본 제시민원인이 제출하지 않아도 되는 서류 (담당공무원이 확인)외국인등록사실증명, 병적증명서, 건강보험자격득실확인서, 국민연금가입자가입증명, 국가기술자격취득사항확인서고등학교졸업증명서참고담당공무원 확인, 본인정보 제공 요구 사항은 본인이 행정정보공동이용에 동의하지 않는 경우 민원인이 제출하여야 합니다.부가정보근거법령정보통신공사업법 (제39조) (제69조 제2항 제4호) (제73조 제6호)정보통신공사업법 시행령 (제39조) (제53조 제2항 제5호) (제57조 제1항)정보통신공사업법 시행에 관한 규정 (제11조 [별지 제30호 서식])제도를 담당하는 기관 :과학기술정보통신부네트워크정책과위 담당부서와 전화번호는 이 민원의 제도를 담당하고 있는 (중앙)행정기관입니다.개별 민원에 대한 문의 사항은 접수·처리기관(관할처리기관)과 연락하시기 바랍니다.정보 변경 내역최근 내용 변경일 : 2024-11-11최근 내용 확인일 : 2024-11-11"
1,https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A08003&CappBizCD=13200000052&tp_seq=01,지문등 사전등록 신청,"서비스 개요신청방법인터넷,방문,FAX,우편,모바일,전화,현장제출 및 기타신청자격법령상의 자격이 있는 자(제3자)처리기간즉시(근무시간 내 3시간)처리기간 계산 방법신청서아동등 사전등록신청서 (실종아동등의 발견 및 유전자검사 등에 관한 규칙 : 별지서식 1호)신청작성예시구비서류구비서류 없음수수료수수료 없음기본정보제공 내용이 민원은 실종아동등의 예방 및 조속한 발견과 복귀를 위하여 아동등의 보호자가 신청하는 경우 아동등의 지문 및 얼굴 등에 관한 정보를 미리 등록하는 민원사무입니다신청시기제한없음후속 조치사항- 경찰청에서는 사전등록 신청에 따라 등록된 지문등정보를 데이터베이스로 구축, 운영할 수 있습니다.- 누구든지 정당한 사유 없이 지문등정보를 실종아동등을 찾기 위한 목적 외로 이용하여서는 아니되며, 이를 위반하여 지문등정보를 실종아동등을 찾기 위한 목적 외로 이용한 경우 2년 이하의 징역 또는 1천만원 이하의 벌금에 처합니다.신청자격아동등의 보호자신청 방법 및 절차1접수경찰서파출소2처리경찰서참고각 기관을 선택하면 해당기관 정보조회가 가능하며, 조회된 접수/처리기관에 대한 실제 민원 접수/처리 가능여부는 해당기관에 확인해 주시기 바랍니다.제출 서류민원인이 제출해야하는 서류없음민원인이 제출하지 않아도 되는 서류 (담당공무원이 확인)주민등록표 등본장애인증명서 (등록대상 아동등이 지적장애인, 자폐성장애인 또는 정신장애인인 경우만 해당합니다)참고담당공무원 확인, 본인정보 제공 요구 사항은 본인이 행정정보공동이용에 동의하지 않는 경우 민원인이 제출하여야 합니다.부가정보온라인신청http://www.safe182.go.kr근거법령실종아동등의 보호 및 지원에 관한 법률 (제7조의2 제1항)실종아동등의 발견 및 유전자검사 등에 관한 규칙 (제3조 제2항)제도를 담당하는 기관 :경찰청청소년보호과 02-3150-2249위 담당부서와 전화번호는 이 민원의 제도를 담당하고 있는 (중앙)행정기관입니다.개별 민원에 대한 문의 사항은 접수·처리기관(관할처리기관)과 연락하시기 바랍니다.정보 변경 내역최근 내용 변경일 : 2024-08-29최근 내용 확인일 : 2024-08-29"
2,https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A09002&CappBizCD=12100000169&tp_seq=,교통·에너지·환경세 미납세(면세) 반출물품 멸실승인(신청),"서비스 개요신청방법방문,우편신청자격누구나 신청 가능처리기간총 5일처리기간 계산 방법신청서교통.에너지.환경세(미납세/면세)반출물품멸실(승인신청서/승인서) (교통ㆍ에너지ㆍ환경세법 시행규칙 : 별지서식 10호)구비서류있음 (하단 참조)수수료수수료 없음기본정보제공 내용이 민원은 교통.에너지.환경세(미납세·면세)반출물품을 화재등으로 멸실한 자가 교통세 징수를 면제 받고자 반출지 관할세무서장 또는 세관장에게 그 승인을 얻고자 신청하는 민원입니다.신청 방법 및 절차1접수세무서2처리세무서참고각 기관을 선택하면 해당기관 정보조회가 가능하며, 조회된 접수/처리기관에 대한 실제 민원 접수/처리 가능여부는 해당기관에 확인해 주시기 바랍니다.제출 서류민원인이 제출해야하는 서류당해 멸실사실증명서참고담당공무원 확인, 본인정보 제공 요구 사항은 본인이 행정정보공동이용에 동의하지 않는 경우 민원인이 제출하여야 합니다.부가정보근거법령교통ㆍ에너지ㆍ환경세법 (제12조제3항)교통ㆍ에너지ㆍ환경세법 시행령 (제18조제1항)교통ㆍ에너지ㆍ환경세법 시행규칙 (제8조제1항및[별지10])제도를 담당하는 기관 :국세청소비세과 (국번없이)126위 담당부서와 전화번호는 이 민원의 제도를 담당하고 있는 (중앙)행정기관입니다.개별 민원에 대한 문의 사항은 접수·처리기관(관할처리기관)과 연락하시기 바랍니다.정보 변경 내역최근 내용 변경일 : 2018-08-10최근 내용 확인일 : 2018-08-10"
3,https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A02002&CappBizCD=14800000063&tp_seq=01,특정토양오염관리대상시설 설치변경(폐쇄)신고,"서비스 개요신청방법인터넷,방문,우편,민원우편신청자격누구나 신청 가능처리기간총 3일처리기간 계산 방법신청서특정토양오염관리대상시설 설치변경(폐쇄)신고서 (토양환경보전법 시행규칙 : 별지서식 5호)신청작성예시구비서류있음 (하단 참조)수수료수수료 없음기본정보제공 내용이 민원은 특정토양오염 관리대상시설의 설치변경 또는 폐쇄신고를 하기 위하여 관할 특별자치도지사, 시장, 군수 및 구청장에게 신청하는 민원사무입니다.신청 방법 및 절차1접수특별자치시장, 특별자치도지사, 시장·군수·구청장2처리특별자치시장, 특별자치도지사, 시장·군수·구청장참고각 기관을 선택하면 해당기관 정보조회가 가능하며, 조회된 접수/처리기관에 대한 실제 민원 접수/처리 가능여부는 해당기관에 확인해 주시기 바랍니다.제출 서류민원인이 제출해야하는 서류특정토양오염관리대상시설 설치신고필증 원본변경(폐쇄)내용을증명하는서류참고담당공무원 확인, 본인정보 제공 요구 사항은 본인이 행정정보공동이용에 동의하지 않는 경우 민원인이 제출하여야 합니다.부가정보온라인신청https://www.gov.go.kr근거법령토양환경보전법 (제12조 제1항)토양환경보전법 시행령 (제6조 제2항)토양환경보전법 시행규칙 (제9조)특정토양오염관리대상시설 관리지침 (제4조)제도를 담당하는 기관 :환경부토양지하수과위 담당부서와 전화번호는 이 민원의 제도를 담당하고 있는 (중앙)행정기관입니다.개별 민원에 대한 문의 사항은 접수·처리기관(관할처리기관)과 연락하시기 바랍니다.정보 변경 내역최근 내용 변경일 : 2025-02-06최근 내용 확인일 : 2025-02-06"
4,htt

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 전체 링크 불러오기
df = pd.read_excel("gov_all_links.xlsx")

def parse_detail_page(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, "html.parser")

        title_tag = soup.select_one("div#pageCont h2")
        title = title_tag.get_text(strip=True) if title_tag else ""

        if soup.select_one("div.contentsWrap.r2n"):
            content_section = soup.select_one("div.contentsWrap.r2n")
            content_texts = [
                tag.get_text(strip=True)
                for tag in content_section.select("h2, div.cont-box, p")
                if tag.get_text(strip=True)
            ]
            content = "\n".join(content_texts)

        elif soup.select_one("div.wrap_col div.left"):
            left = soup.select_one("div.wrap_col div.left")
            content = left.get_text(strip=True)

        else:
            content = "알 수 없는 레이아웃"

        return {"URL": url, "제목": title, "내용": content}
    
    except Exception as e:
        return {"URL": url, "제목": "", "내용": f"오류: {e}"}

# 전체 링크 처리
results = []
for i, url in enumerate(df["URL"]):
    print(f"{i+1}/{len(df)} 처리 중: {url}")
    data = parse_detail_page(url)
    results.append(data)
    time.sleep(0.5)  # 서버 부하 방지

# 저장
full_df = pd.DataFrame(results)
full_df.to_excel("gov_service_details_full.xlsx", index=False)
print("완료되었습니다.")

1/10263 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A03004&CappBizCD=15701000076&tp_seq=
2/10263 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A08003&CappBizCD=13200000052&tp_seq=01
3/10263 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A09002&CappBizCD=12100000169&tp_seq=
4/10263 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A02002&CappBizCD=14800000063&tp_seq=01
5/10263 처리 중: https://www.gov.kr/portal/service/serviceInfo/403000000147
6/10263 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A03007&CappBizCD=15200000480&tp_seq=01
7/10263 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A03006&CappBizCD=14100000069&tp_seq=01
8/10263 처리 중: https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A09002&CappBizCD=12200000235&tp_seq=01
9/10263 처리 중: https://www.gov.kr/portal/service/serviceInfo/508000000565
10/10263 처리 중: https://www.gov.kr/portal/service/serviceInfo/356000000232
11/10263 처리 중: https://www.gov.kr/portal/s

In [9]:
full_df

,URL,제목,내용
0,https://www.gov.kr/mw/AA020InfoCappView.do?Hig...,,알 수 없는 레이아웃
1,https://www.gov.kr/mw/AA020InfoCappView.do?Hig...,지문등 사전등록 신청,"서비스 개요신청방법인터넷,방문,FAX,우편,모바일,전화,현장제출 및 기타신청자격법령..."
2,https://www.gov.kr/mw/AA020InfoCappView.do?Hig...,교통·에너지·환경세 미납세(면세) 반출물품 멸실승인(신청),"서비스 개요신청방법방문,우편신청자격누구나 신청 가능처리기간총 5일처리기간 계산 방법..."
3,https://www.gov.kr/mw/AA020InfoCappView.do?Hig...,특정토양오염관리대상시설 설치변경(폐쇄)신고,"서비스 개요신청방법인터넷,방문,우편,민원우편신청자격누구나 신청 가능처리기간총 3일처..."
4,https://www.gov.kr/portal/service/serviceInfo/...,노인 버스 무료승차 지원,어떤 내용인지 궁금하시죠?\n지원형태현금지원내용○대상: 65세이상 의왕시 거주 시민...
...,...,...,...
10258,https://www.gov.kr/mw/AA020InfoCappView.do?Hig...,"가축분뇨 배출시설 및 처리시설의 (개선,조치,사용중지,폐쇄)명령 이행 보고","서비스 개요신청방법인터넷,방문,우편신청자격누구나 신청 가능처리기간총 4일처리기간 계..."
10259,https://www.gov.kr/mw/AA020InfoCappView.do?Hig...,지역농업협동조합 설립인가 신청,"서비스 개요신청방법방문,우편신청자격누구나 신청 가능처리기간총 60일처리기간 계산 방..."
10260,https://www.gov.kr/mw/AA020InfoCappView.do?Hig...,기초원재료납세증명서 발급신청,서비스 개요신청방법인터넷신청자격누구나 신청 가능처리기간총 1일처리기간 계산 방법신청...
10261,https://www.gov.kr/portal/service/serviceInfo/...,정신건강복지센터 운영(아동·청소년 정신보건사업 포함),○ 인식개선을 통한 정신건강서비스 이용 제고\r\n○ 정신건강 문제 조기발견 및 개...


In [10]:
full_df1 = full_df

In [11]:
# 특수문자 제거
full_df1["내용"] = full_df1["내용"].str.replace(r"[\n\r\t]", "", regex=True)

# 고정 텍스트 제거
제거문구 = "근거법령소관기관최종수정일분야별 서비스 Best 5기관별 서비스 Best 5"
full_df1["내용"] = full_df1["내용"].str.replace(제거문구, "", regex=False)

In [12]:
# 이 페이지에 만족하시나요?네아니오 문구 이후 제거

# 기준 문구 지정
cut_point = "이 페이지에 만족하시나요?네아니오"

# 문구 이후 제거
full_df1["내용"] = full_df1["내용"].apply(
    lambda x: x.split(cut_point)[0] if isinstance(x, str) and cut_point in x else x
)

In [22]:
import re

def clean_invalid_unicode(s):
    if isinstance(s, str):
        # 서러게이트 문자 제거
        s = re.sub(r'[\ud800-\udfff]', '', s)
        # 제어 문자 제거
        s = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', s)
    return s

# 복사 및 정제
full_df2 = full_df1.copy()
for col in full_df2.columns:
    full_df2[col] = full_df2[col].apply(clean_invalid_unicode)

# CSV 저장
full_df2.to_csv("full_df2.csv", index=False, encoding="utf-8-sig")

In [24]:
import pandas as pd

# 긴 열 생략 없이 전체 표시
pd.set_option('display.max_colwidth', None)

# 전체 열 표시 (생략 방지)
pd.set_option('display.max_columns', None)

# 행 생략 방지 (원한다면)
pd.set_option('display.max_rows', None)

In [25]:
import pandas as pd

full_df2_loaded = pd.read_csv("full_df2.csv", encoding="utf-8-sig")
full_df2_loaded.head()  # 데이터 일부 확인

,URL,제목,내용
0,https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A03004&CappBizCD=15701000076&tp_seq=,NaN,알 수 없는 레이아웃
1,https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A08003&CappBizCD=13200000052&tp_seq=01,지문등 사전등록 신청,"서비스 개요신청방법인터넷,방문,FAX,우편,모바일,전화,현장제출 및 기타신청자격법령상의 자격이 있는 자(제3자)처리기간즉시(근무시간 내 3시간)처리기간 계산 방법신청서아동등 사전등록신청서 (실종아동등의 발견 및 유전자검사 등에 관한 규칙 : 별지서식 1호)신청작성예시구비서류구비서류 없음수수료수수료 없음기본정보제공 내용이 민원은 실종아동등의 예방 및 조속한 발견과 복귀를 위하여 아동등의 보호자가 신청하는 경우 아동등의 지문 및 얼굴 등에 관한 정보를 미리 등록하는 민원사무입니다신청시기제한없음후속 조치사항- 경찰청에서는 사전등록 신청에 따라 등록된 지문등정보를 데이터베이스로 구축, 운영할 수 있습니다.- 누구든지 정당한 사유 없이 지문등정보를 실종아동등을 찾기 위한 목적 외로 이용하여서는 아니되며, 이를 위반하여 지문등정보를 실종아동등을 찾기 위한 목적 외로 이용한 경우 2년 이하의 징역 또는 1천만원 이하의 벌금에 처합니다.신청자격아동등의 보호자신청 방법 및 절차1접수경찰서파출소2처리경찰서참고각 기관을 선택하면 해당기관 정보조회가 가능하며, 조회된 접수/처리기관에 대한 실제 민원 접수/처리 가능여부는 해당기관에 확인해 주시기 바랍니다.제출 서류민원인이 제출해야하는 서류없음민원인이 제출하지 않아도 되는 서류 (담당공무원이 확인)주민등록표 등본장애인증명서 (등록대상 아동등이 지적장애인, 자폐성장애인 또는 정신장애인인 경우만 해당합니다)참고담당공무원 확인, 본인정보 제공 요구 사항은 본인이 행정정보공동이용에 동의하지 않는 경우 민원인이 제출하여야 합니다.부가정보온라인신청http://www.safe182.go.kr근거법령실종아동등의 보호 및 지원에 관한 법률 (제7조의2 제1항)실종아동등의 발견 및 유전자검사 등에 관한 규칙 (제3조 제2항)제도를 담당하는 기관 :경찰청청소년보호과 02-3150-2249위 담당부서와 전화번호는 이 민원의 제도를 담당하고 있는 (중앙)행정기관입니다.개별 민원에 대한 문의 사항은 접수·처리기관(관할처리기관)과 연락하시기 바랍니다.정보 변경 내역최근 내용 변경일 : 2024-08-29최근 내용 확인일 : 2024-08-29"
2,https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A09002&CappBizCD=12100000169&tp_seq=,교통·에너지·환경세 미납세(면세) 반출물품 멸실승인(신청),"서비스 개요신청방법방문,우편신청자격누구나 신청 가능처리기간총 5일처리기간 계산 방법신청서교통.에너지.환경세(미납세/면세)반출물품멸실(승인신청서/승인서) (교통ㆍ에너지ㆍ환경세법 시행규칙 : 별지서식 10호)구비서류있음 (하단 참조)수수료수수료 없음기본정보제공 내용이 민원은 교통.에너지.환경세(미납세·면세)반출물품을 화재등으로 멸실한 자가 교통세 징수를 면제 받고자 반출지 관할세무서장 또는 세관장에게 그 승인을 얻고자 신청하는 민원입니다.신청 방법 및 절차1접수세무서2처리세무서참고각 기관을 선택하면 해당기관 정보조회가 가능하며, 조회된 접수/처리기관에 대한 실제 민원 접수/처리 가능여부는 해당기관에 확인해 주시기 바랍니다.제출 서류민원인이 제출해야하는 서류당해 멸실사실증명서참고담당공무원 확인, 본인정보 제공 요구 사항은 본인이 행정정보공동이용에 동의하지 않는 경우 민원인이 제출하여야 합니다.부가정보근거법령교통ㆍ에너지ㆍ환경세법 (제12조제3항)교통ㆍ에너지ㆍ환경세법 시행령 (제18조제1항)교통ㆍ에너지ㆍ환경세법 시행규칙 (제8조제1항및[별지10])제도를 담당하는 기관 :국세청소비세과 (국번없이)126위 담당부서와 전화번호는 이 민원의 제도를 담당하고 있는 (중앙)행정기관입니다.개별 민원에 대한 문의 사항은 접수·처리기관(관할처리기관)과 연락하시기 바랍니다.정보 변경 내역최근 내용 변경일 : 2018-08-10최근 내용 확인일 : 2018-08-10"
3,https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A02002&CappBizCD=14800000063&tp_seq=01,특정토양오염관리대상시설 설치변경(폐쇄)신고,"서비스 개요신청방법인터넷,방문,우편,민원우편신청자격누구나 신청 가능처리기간총 3일처리기간 계산 방법신청서특정토양오염관리대상시설 설치변경(폐쇄)신고서 (토양환경보전법 시행규칙 : 별지서식 5호)신청작성예시구비서류있음 (하단 참조)수수료수수료 없음기본정보제공 내용이 민원은 특정토양오염 관리대상시설의 설치변경 또는 폐쇄신고를 하기 위하여 관할 특별자치도지사, 시장, 군수 및 구청장에게 신청하는 민원사무입니다.신청 방법 및 절차1접수특별자치시장, 특별자치도지사, 시장·군수·구청장2처리특별자치시장, 특별자치도지사, 시장·군수·구청장참고각 기관을 선택하면 해당기관 정보조회가 가능하며, 조회된 접수/처리기관에 대한 실제 민원 접수/처리 가능여부는 해당기관에 확인해 주시기 바랍니다.제출 서류민원인이 제출해야하는 서류특정토양오염관리대상시설 설치신고필증 원본변경(폐쇄)내용을증명하는서류참고담당공무원 확인, 본인정보 제공 요구 사항은 본인이 행정정보공동이용에 동의하지 않는 경우 민원인이 제출하여야 합니다.부가정보온라인신청https://www.gov.go.kr근거법령토양환경보전법 (제12조 제1항)토양환경보전법 시행령 (제6조 제2항)토양환경보전법 시행규칙 (제9조)특정토양오염관리대상시설 관리지침 (제4조)제도를 담당하는 기관 :환경부토양지하수과위 담당부서와 전화번호는 이 민원의 제도를 담당하고 있는 (중앙)행정기관입니다.개별 민원에 대한 문의 사항은 접수·처리기관(관할처리기관)과 연락하시기 바랍니다.정보 변경 내역최근 내용 변경일 : 2025-02-06최근 내용 확인일 : 2025-02-06"
4,https://www.gov.kr/portal/service/serviceInfo/403000000147,노인 버스 무료승차 지원,"어떤 내용인지 궁금하시죠?지원형태현금지원내용○대상: 65세이상 의왕시 거주 시민( 해당년도 생일 지나신 분)○지원내용: 의왕시 정차, 시내, 마을, 광역 버스 요금(환승비포함) 지원(시원, 공항버스 제외)○지원금액: 연간 20만원(분기별 5만원)○ 지원방법: 충전 선불 교통카드 이용 ⇒ 분기별 실사용료 현금 지급○ 신청기간: 상시발급지원형태지원내용이런 분들께 해당합니다.지원대상65세이상 의왕시 거주 노인(해당년도 생일 지나신 분)지원대상이용 방법은 이렇습니다.절차/방법신분증, 기존G-PASS카드(기 발급받은 경우), 농협통장(소지자에 한함) 지참하여 관내 농협 직접 방문, 즉시 발급, 충전후 바로 사용(반드시 본인발급)구비서류신분증, 기존G-PASS카드(기 발급받은 경우), 농협통장(소지자에 한함) 지참하여 관내 농협 직접 방문, 즉시 발급, 충전후 바로 사용(반드시 본인발급)절차/방법구비서류※ 자세한 사항은 해당 기관의 홈페이지 또는 문의처를 통해 확인하시기 바랍니다.참고정보입니다.근거법령[자치법규] 의왕시 노인 버스 무료승차 지원 조례소관기관경기도 의왕시 안전환경교통국 대중교통과최종수정일2025.05.07"
